In [1]:
import math
import os
import uuid
import time

from matplotlib import cm
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
import itertools
import tensorflow as tf

# Assuming Waymo Open Dataset protos and utilities are correctly installed
try:
    from google.protobuf import text_format
    from waymo_open_dataset.protos import scenario_pb2
    # The metrics/config_util_py are for motion metrics, not parsing the core data itself
    # from waymo_open_dataset.metrics.ops import py_metrics_ops # Not strictly needed for this task
    # from waymo_open_dataset.metrics.python import config_util_py as config_util # Not strictly needed
    # from waymo_open_dataset.protos import motion_metrics_pb2 # Not strictly needed
    print("Waymo Open Dataset modules imported successfully.")
except ImportError as e:
    print(f"Error importing Waymo modules: {e}")
    print("Please ensure 'waymo-open-dataset' is installed: pip install waymo-open-dataset")
    exit()

# Define input and output directories
INPUT_DIR = '/home/hamdarlab/Desktop/Waymo/data20s/' # Adjust if your path changed
OUTPUT_DIR = '/home/hamdarlab/Desktop/Waymo/data20s/plots_scenarios_with_lanes/' # Adjusted output

# Create the output directory if it doesn't exist
try:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Output directory '{OUTPUT_DIR}' ensured.")
except OSError as e:
    print(f"Error creating output directory '{OUTPUT_DIR}': {e}")
    print("Please check directory permissions.")
    exit()

# --- Helper functions for distance calculation ---
def point_to_segment_distance(px, py, x1, y1, x2, y2):
    """Calculate the shortest distance from point (px, py) to line segment (x1,y1)-(x2,y2)."""
    line_mag_sq = (x2 - x1)**2 + (y2 - y1)**2
    if line_mag_sq == 0:  # Segment is a point
        return math.sqrt((px - x1)**2 + (py - y1)**2)

    u = ((px - x1) * (x2 - x1) + (py - y1) * (y2 - y1)) / line_mag_sq

    if u < 0:
        closest_x, closest_y = x1, y1
    elif u > 1:
        closest_x, closest_y = x2, y2
    else:
        closest_x = x1 + u * (x2 - x1)
        closest_y = y1 + u * (y2 - y1)
    
    return math.sqrt((px - closest_x)**2 + (py - closest_y)**2)

def point_to_polyline_distance(point_x, point_y, polyline_points):
    """Calculate the minimum distance from a point to a polyline (series of segments)."""
    min_dist = float('inf')
    if polyline_points.shape[0] == 0: # No points in polyline
        return min_dist
    if polyline_points.shape[0] == 1: # Polyline is a single point
        return math.sqrt((point_x - polyline_points[0,0])**2 + (point_y - polyline_points[0,1])**2)

    for i in range(len(polyline_points) - 1):
        x1, y1 = polyline_points[i]
        x2, y2 = polyline_points[i+1]
        dist = point_to_segment_distance(point_x, point_y, x1, y1, x2, y2)
        if dist < min_dist:
            min_dist = dist
    return min_dist

# Helper functions for plotting setup
def create_figure_and_axes(size_pixels):
  fig, ax = plt.subplots(1, 1, num=uuid.uuid4())
  dpi = 100
  size_inches = size_pixels / dpi
  fig.set_size_inches([size_inches, size_inches])
  fig.set_dpi(dpi)
  fig.set_facecolor('white')
  ax.set_facecolor('white')
  ax.xaxis.label.set_color('black')
  ax.tick_params(axis='x', colors='black')
  ax.yaxis.label.set_color('black')
  ax.tick_params(axis='y', colors='black')
  fig.set_tight_layout(True)
  ax.grid(False)
  return fig, ax


def fig_canvas_image(fig):
  fig.subplots_adjust(
      left=0.08, bottom=0.08, right=0.98, top=0.98, wspace=0.0, hspace=0.0)
  fig.canvas.draw()
  data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  return data.reshape(fig.canvas.get_width_height()[::-1] + (3,))


def get_colormap(num_agents):
  colors = cm.get_cmap('jet', num_agents)
  colors = colors(range(num_agents))
  np.random.shuffle(colors)
  return colors


def get_viewport(all_states, all_states_mask):
  valid_states = all_states[all_states_mask]
  all_y = valid_states[..., 1]
  all_x = valid_states[..., 0]

  if valid_states.size == 0:
      return 0, 0, 10 # Default to a small viewport if no valid states

  center_y = (np.max(all_y) + np.min(all_y)) / 2
  center_x = (np.max(all_x) + np.min(all_x)) / 2

  range_y = np.ptp(all_y) # Peak-to-peak range
  range_x = np.ptp(all_x)

  width = max(range_y, range_x, 10) # Ensure minimum width

  return center_y, center_x, width


# --- MODIFIED visualize_scenario_static function ---
def visualize_scenario_static(
    scenario_proto,
    size_pixels=1000,
):
    if not scenario_proto.tracks:
        print(f"    Scenario {scenario_proto.scenario_id}: No tracks found. Skipping plot.")
        return None

    total_steps = 0
    for track in scenario_proto.tracks:
        if track.states:
            total_steps = len(track.states)
            break
    
    if total_steps == 0:
        print(f"    Scenario {scenario_proto.scenario_id}: No track states found. Skipping plot.")
        return None

    vis_current_time_idx = min(99, total_steps - 1) 

    # Extract agent data
    _all_agent_states_list = []
    _all_agent_masks_list = []
    _agent_object_types_list = []
    _agent_track_ids_list = [] # To store original track.id

    for i, track in enumerate(scenario_proto.tracks):
        if len(track.states) != total_steps:
            print(f"    WARNING: Scenario {scenario_proto.scenario_id}, Track {track.id} has {len(track.states)} states, expected {total_steps}. Skipping this track for data extraction.")
            continue
        
        track_states_np = np.zeros((total_steps, 2), dtype=np.float32)
        track_masks_np = np.zeros(total_steps, dtype=bool)
        
        for array_idx, state in enumerate(track.states):
            track_states_np[array_idx, 0] = state.center_x
            track_states_np[array_idx, 1] = state.center_y
            track_masks_np[array_idx] = state.valid

        _all_agent_states_list.append(track_states_np)
        _all_agent_masks_list.append(track_masks_np)
        _agent_object_types_list.append(track.object_type)
        _agent_track_ids_list.append(track.id)
            
    if not _all_agent_states_list:
        print(f"    Scenario {scenario_proto.scenario_id}: No valid agent tracks found after filtering. Skipping plot.")
        return None

    all_agent_states = np.array(_all_agent_states_list)
    all_agent_masks = np.array(_all_agent_masks_list)
    agent_object_types = np.array(_agent_object_types_list)
    # _agent_track_ids_list is now correctly aligned with the rows of all_agent_states
    
    num_agents = all_agent_states.shape[0]

    past_states = all_agent_states[:, :vis_current_time_idx, :]
    past_states_mask = all_agent_masks[:, :vis_current_time_idx]
    current_states = all_agent_states[:, vis_current_time_idx : vis_current_time_idx + 1, :]
    current_states_mask = all_agent_masks[:, vis_current_time_idx : vis_current_time_idx + 1]
    future_states = all_agent_states[:, vis_current_time_idx + 1 :, :]
    future_states_mask = all_agent_masks[:, vis_current_time_idx + 1 :]

    # Extract roadgraph data and lane data
    roadgraph_xyz_list = []
    lanes_data = {}  # Store {lane_id: np.array([[x1,y1], [x2,y2], ...])}
    for map_feature in scenario_proto.map_features:
        feature_id = map_feature.id # Common ID for all map features
        if map_feature.HasField('lane'):
            points = []
            for point in map_feature.lane.polyline:
                roadgraph_xyz_list.append([point.x, point.y, point.z])
                points.append((point.x, point.y))
            if points:
                lanes_data[feature_id] = np.array(points)
        elif map_feature.HasField('road_line'):
            for point in map_feature.road_line.polyline:
                roadgraph_xyz_list.append([point.x, point.y, point.z])
        elif map_feature.HasField('road_edge'):
            for point in map_feature.road_edge.polyline:
                roadgraph_xyz_list.append([point.x, point.y, point.z])
        # Add other features if needed for full roadgraph visualization
        elif map_feature.HasField('stop_sign'):
            roadgraph_xyz_list.append([map_feature.stop_sign.position.x, map_feature.stop_sign.position.y, map_feature.stop_sign.position.z])
        elif map_feature.HasField('crosswalk'):
             for point in map_feature.crosswalk.polygon:
                 roadgraph_xyz_list.append([point.x, point.y, point.z])


    roadgraph_xyz = np.array(roadgraph_xyz_list, dtype=np.float32) if roadgraph_xyz_list else None

    # --- Agent to Lane Assignment ---
    agent_current_lane_ids = {} # {track_id: assigned_lane_id or None}
    LANE_ASSIGNMENT_THRESHOLD_METERS = 2.5 # Max perpendicular distance to lane centerline

    print(f"  Scenario {scenario_proto.scenario_id}: Assigning agents to lanes...")
    for agent_array_idx in range(num_agents):
        track_id = _agent_track_ids_list[agent_array_idx]
        assigned_lane_id_for_agent = None # Default to no lane

        if current_states_mask[agent_array_idx, 0]: # If agent has a valid current state
            agent_x = current_states[agent_array_idx, 0, 0]
            agent_y = current_states[agent_array_idx, 0, 1]
            
            min_dist_to_lane_center = float('inf')
            
            if not lanes_data:
                agent_current_lane_ids[track_id] = None
                continue

            for lane_id, lane_polyline_pts in lanes_data.items():
                if lane_polyline_pts.shape[0] < 2: # Need at least 2 points for a segment
                    continue
                
                dist = point_to_polyline_distance(agent_x, agent_y, lane_polyline_pts)
                
                if dist < min_dist_to_lane_center:
                    min_dist_to_lane_center = dist
                    assigned_lane_id_for_agent = lane_id
            
            if min_dist_to_lane_center <= LANE_ASSIGNMENT_THRESHOLD_METERS:
                agent_current_lane_ids[track_id] = assigned_lane_id_for_agent
                print(f"    Agent {track_id} assigned to Lane {assigned_lane_id_for_agent} (distance: {min_dist_to_lane_center:.2f}m)")
            else:
                agent_current_lane_ids[track_id] = None # Not close enough to any lane
                # print(f"    Agent {track_id} not on any lane (closest: Lane {assigned_lane_id_for_agent}, dist: {min_dist_to_lane_center:.2f}m)")
        else:
            agent_current_lane_ids[track_id] = None # No valid current state for this agent

    # --- Plotting ---
    color_map = get_colormap(num_agents)
    all_states_for_viewport = np.concatenate([past_states, current_states, future_states], axis=1)
    all_states_mask_for_viewport = np.concatenate(
        [past_states_mask, current_states_mask.reshape(num_agents, -1), future_states_mask], axis=1) # Reshape current_states_mask

    center_y, center_x, width = get_viewport(all_states_for_viewport, all_states_mask_for_viewport)
    fig, ax = create_figure_and_axes(size_pixels=size_pixels)

    # Plot roadgraph (all map features)
    if roadgraph_xyz is not None and roadgraph_xyz.size > 0:
        rg_pts = roadgraph_xyz[:, :2].T
        ax.plot(rg_pts[0, :], rg_pts[1, :], 'k.', alpha=0.2, ms=1.5) # Made fainter
    else:
        print(f"    Note: Scenario {scenario_proto.scenario_id}: No roadgraph data for general plot.")

    # Plot Lane IDs on the map
    if lanes_data:
        for lane_id, polyline_pts_array in lanes_data.items():
            if polyline_pts_array.shape[0] > 0:
                text_pos_idx = polyline_pts_array.shape[0] // 2
                text_x, text_y = polyline_pts_array[text_pos_idx]
                ax.text(text_x + 0.5, text_y + 0.5, f'L:{lane_id}', color='darkslategray', fontsize=5, ha='center', va='bottom', zorder=3, alpha=0.7)

    vehicle_marker = 'o'
    other_marker = '*'
    marker_size = 20 
    star_marker_size = 40
    line_width = 1
    alpha_value = 0.7

    for agent_idx in range(num_agents):
        agent_color = color_map[agent_idx]
        current_marker = vehicle_marker
        current_marker_size = marker_size
        if agent_object_types[agent_idx] != scenario_pb2.Track.ObjectType.TYPE_VEHICLE:
            current_marker = other_marker
            current_marker_size = star_marker_size
        
        # Plot past
        past_x = past_states[agent_idx, :, 0][past_states_mask[agent_idx, :]]
        past_y = past_states[agent_idx, :, 1][past_states_mask[agent_idx, :]]
        ax.scatter(past_x, past_y, marker=current_marker, linewidths=line_width, color=agent_color, alpha=alpha_value*0.5, s=current_marker_size*0.8, zorder=4)

        # Plot current
        if current_states_mask[agent_idx, 0]:
            current_x_val = current_states[agent_idx, 0, 0]
            current_y_val = current_states[agent_idx, 0, 1]
            ax.scatter(current_x_val, current_y_val, marker=current_marker, linewidths=line_width, color=agent_color, alpha=alpha_value, s=current_marker_size * 1.5, zorder=5, edgecolors='black',linewidth=0.5)
            # Optionally, annotate agent with its ID or assigned lane ID
            # track_id = _agent_track_ids_list[agent_idx]
            # assigned_lane = agent_current_lane_ids.get(track_id)
            # if assigned_lane is not None:
            #     ax.text(current_x_val, current_y_val + 1, f"A:{track_id}\nL:{assigned_lane}", color=agent_color, fontsize=5, zorder=6)
            # else:
            #     ax.text(current_x_val, current_y_val + 1, f"A:{track_id}", color=agent_color, fontsize=5, zorder=6)


        # Plot future
        future_x = future_states[agent_idx, :, 0][future_states_mask[agent_idx, :]]
        future_y = future_states[agent_idx, :, 1][future_states_mask[agent_idx, :]]
        ax.scatter(future_x, future_y, marker=current_marker, linewidths=line_width, color=agent_color, alpha=alpha_value*0.7, s=current_marker_size, zorder=4)

    ax.set_title(f'Scenario ID: {scenario_proto.scenario_id} - Agent Trajectories & Lane IDs (Current @ T={vis_current_time_idx})')
    plot_range = max(10, width * 1.05) # Adjusted viewport slightly
    ax.axis([
        -plot_range / 2 + center_x, plot_range / 2 + center_x, 
        -plot_range / 2 + center_y, plot_range / 2 + center_y
    ])
    ax.set_aspect('equal')

    image = fig_canvas_image(fig)
    plt.close(fig)
    return image


# --- Main script to process all files ---
print(f"\n--- Starting Plot Generation ---")
print(f"Checking input directory: {INPUT_DIR}")
print(f"Plots will be saved to: {OUTPUT_DIR}")

try:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Output directory '{OUTPUT_DIR}' ensured.")
except OSError as e:
    print(f"Error creating output directory '{OUTPUT_DIR}': {e}")
    print("Please check directory permissions.")
    exit()

tfrecord_files = []
for root, _, files in os.walk(INPUT_DIR):
    for f in files:
        if '.tfrecord' in f: # More robust check for tfrecord files
            tfrecord_files.append(os.path.join(root, f))

tfrecord_files.sort()

if not tfrecord_files:
    print(f"\nERROR: No TFRecord files found in '{INPUT_DIR}'.")
    print("Please double-check the 'INPUT_DIR' path and the file naming convention (e.g., '.tfrecord').")
else:
    print(f"\nFound {len(tfrecord_files)} TFRecord files to process.")
    for k, found_file in enumerate(tfrecord_files[:min(5, len(tfrecord_files))]): # Print first 5 or fewer
        print(f"  Found file {k+1}: {os.path.basename(found_file)}")
    if len(tfrecord_files) > 5:
        print("  ...")

    for i, full_filepath in enumerate(tfrecord_files):
        filename = os.path.basename(full_filepath)
        base_output_name = os.path.splitext(filename)[0] # Remove .tfrecord
        
        # It seems the original code might produce very long filenames if base_output_name already contains extensions.
        # E.g. training_20s.tfrecord-00000-of-01000 -> training_20s.tfrecord-00000-of-01000_scenario_id.png
        # This is generally fine.

        output_filepath_prefix = os.path.join(OUTPUT_DIR, base_output_name)

        print(f"\n[{i+1}/{len(tfrecord_files)}] Processing '{filename}'...")
        try:
            dataset = tf.data.TFRecordDataset(full_filepath, compression_type='')
            successful_plots_in_file = 0
            scenarios_in_file = 0

            for j, data_bytes in enumerate(dataset.as_numpy_iterator()):
                scenarios_in_file +=1
                scenario_id_for_log = "N/A"
                try:
                    scenario = scenario_pb2.Scenario()
                    scenario.ParseFromString(data_bytes)
                    scenario_id_for_log = scenario.scenario_id
                    # print(f"  Parsing scenario {j} (ID: {scenario.scenario_id})...") # Moved inside try
                    
                    plot_image = visualize_scenario_static(scenario)
                    
                    if plot_image is None or plot_image.size == 0:
                        # print(f"  - Scenario {scenario_id_for_log}: Plot image was not generated or empty. Skipping save.")
                        continue

                    scenario_output_filepath = f"{output_filepath_prefix}_scenario_{scenario.scenario_id}.png"
                    plt.imsave(scenario_output_filepath, plot_image)
                    # print(f"  - Saved plot for scenario {scenario.scenario_id} to '{scenario_output_filepath}'")
                    successful_plots_in_file += 1

                except Exception as e:
                    print(f"  - WARNING: An error occurred while processing scenario {j} (ID: {scenario_id_for_log}) in '{filename}': {e}")
                    # import traceback
                    # traceback.print_exc() # Uncomment for detailed error trace
                    continue
            
            if scenarios_in_file == 0:
                print(f"  - No scenarios found or iterated in '{filename}'.")
            elif successful_plots_in_file == 0:
                print(f"  - Processed {scenarios_in_file} scenarios in '{filename}', but none were successfully plotted.")
            else:
                print(f"  - Successfully plotted {successful_plots_in_file} out of {scenarios_in_file} scenarios from '{filename}'.")

        except tf.errors.DataLossError as e:
            print(f"  - ERROR: Data loss or corruption detected in '{filename}': {e}. Skipping this file.")
        except Exception as e:
            print(f"  - ERROR: General error loading or processing file '{filename}': {e}. Skipping this file.")
            # import traceback
            # traceback.print_exc() # Uncomment for detailed error trace


print("\n--- Plotting process complete. ---")
print(f"Check '{OUTPUT_DIR}' for generated images.")

2025-05-24 14:55:43.303239: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-24 14:55:43.342741: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-24 14:55:43.343706: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-24 14:55:44.093339: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Waymo Open Dataset modules imported successfully.
Output directory '/home/hamdarlab/Desktop/Waymo/data20s/plots_scenarios_with_lanes/' ensured.

--- Starting Plot Generation ---
Checking input directory: /home/hamdarlab/Desktop/Waymo/data20s/
Plots will be saved to: /home/hamdarlab/Desktop/Waymo/data20s/plots_scenarios_with_lanes/
Output directory '/home/hamdarlab/Desktop/Waymo/data20s/plots_scenarios_with_lanes/' ensured.

Found 20 TFRecord files to process.
  Found file 1: training_20s.tfrecord-00000-of-01000
  Found file 2: training_20s.tfrecord-00001-of-01000
  Found file 3: training_20s.tfrecord-00002-of-01000
  Found file 4: training_20s.tfrecord-00003-of-01000
  Found file 5: training_20s.tfrecord-00004-of-01000
  ...

[1/20] Processing 'training_20s.tfrecord-00000-of-01000'...
  Scenario 41600af30ab8cc55: Assigning agents to lanes...


2025-05-24 14:55:45.405529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


    Agent 128 assigned to Lane 184 (distance: 0.08m)
    Agent 129 assigned to Lane 389 (distance: 0.14m)
    Agent 130 assigned to Lane 180 (distance: 0.06m)
    Agent 131 assigned to Lane 183 (distance: 0.18m)
    Agent 133 assigned to Lane 231 (distance: 0.11m)
    Agent 135 assigned to Lane 180 (distance: 0.12m)
    Agent 136 assigned to Lane 183 (distance: 0.28m)
    Agent 137 assigned to Lane 389 (distance: 0.12m)
    Agent 138 assigned to Lane 391 (distance: 0.70m)
    Agent 139 assigned to Lane 184 (distance: 0.09m)
    Agent 140 assigned to Lane 391 (distance: 0.05m)
    Agent 141 assigned to Lane 184 (distance: 0.29m)
    Agent 142 assigned to Lane 392 (distance: 0.08m)
    Agent 144 assigned to Lane 180 (distance: 0.40m)
    Agent 145 assigned to Lane 180 (distance: 0.03m)
    Agent 146 assigned to Lane 184 (distance: 0.15m)
    Agent 919 assigned to Lane 347 (distance: 0.06m)
    Agent 105 assigned to Lane 184 (distance: 0.02m)
    Agent 106 assigned to Lane 180 (distance: 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f237bb08b50>>
Traceback (most recent call last):
  File "/home/hamdarlab/anaconda3/envs/waymo-env/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


...

/tmp/ipykernel_8625/1875686489.py:97: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.canvas.draw()


  Scenario 4bf1d627f1771287: Assigning agents to lanes...
    Agent 0 assigned to Lane 135 (distance: 0.14m)
    Agent 1 assigned to Lane 155 (distance: 0.04m)
    Agent 2 assigned to Lane 155 (distance: 0.17m)
    Agent 3 assigned to Lane 172 (distance: 0.05m)
    Agent 4 assigned to Lane 179 (distance: 0.33m)
    Agent 5 assigned to Lane 140 (distance: 0.12m)
    Agent 6 assigned to Lane 132 (distance: 0.29m)
    Agent 7 assigned to Lane 155 (distance: 0.29m)
    Agent 10 assigned to Lane 167 (distance: 0.68m)
    Agent 11 assigned to Lane 135 (distance: 0.25m)
    Agent 12 assigned to Lane 200 (distance: 0.13m)
    Agent 13 assigned to Lane 146 (distance: 0.11m)
    Agent 16 assigned to Lane 201 (distance: 0.07m)
    Agent 18 assigned to Lane 146 (distance: 0.15m)
    Agent 23 assigned to Lane 155 (distance: 0.20m)
    Agent 31 assigned to Lane 149 (distance: 0.09m)
    Agent 34 assigned to Lane 201 (distance: 0.19m)
    Agent 36 assigned to Lane 150 (distance: 0.66m)
    Agent 42 a

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x7f2330ed7250>
Traceback (most recent call last):
  File "/home/hamdarlab/anaconda3/envs/waymo-env/lib/python3.10/weakref.py", line 106, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt: 


  Scenario 44dc56e65fc65a82: Assigning agents to lanes...
    Agent 650 assigned to Lane 412 (distance: 0.43m)
    Agent 655 assigned to Lane 444 (distance: 0.29m)
    Agent 671 assigned to Lane 384 (distance: 0.29m)
    Agent 672 assigned to Lane 437 (distance: 0.07m)
    Agent 675 assigned to Lane 435 (distance: 0.11m)
    Agent 677 assigned to Lane 424 (distance: 0.08m)
    Agent 680 assigned to Lane 435 (distance: 0.29m)
    Agent 688 assigned to Lane 386 (distance: 0.30m)
    Agent 695 assigned to Lane 425 (distance: 0.02m)
    Agent 880 assigned to Lane 407 (distance: 0.21m)
  Scenario 7ac22c9e42d05c79: Assigning agents to lanes...
    Agent 0 assigned to Lane 155 (distance: 0.16m)
    Agent 1 assigned to Lane 153 (distance: 0.09m)
    Agent 2 assigned to Lane 164 (distance: 0.16m)
    Agent 3 assigned to Lane 166 (distance: 0.95m)
    Agent 4 assigned to Lane 169 (distance: 0.02m)
    Agent 5 assigned to Lane 168 (distance: 0.42m)
    Agent 6 assigned to Lane 153 (distance: 0.05


KeyboardInterrupt

